In [1]:
import pandas as pd

In [2]:
review = pd.read_excel('data/data_all.xlsx')

In [3]:
review.head()

,site,hotel,score,review,date,star,length,review_spell_check,helpful,attitude
0,야놀자,신라스테이 광화문,10.0,깔끔합니다,2019. 03. 30,5.0,5,깔끔합니다,0,2
1,야놀자,신라스테이 광화문,6.0,다니기는 참 좋은데 청소를 아예 안 하시네요,2020. 09. 13,3.0,24,다니기는 참 좋은데 청소를 아예 안 하시네요,1,0
2,야놀자,나인트리 프리미어 명동2,10.0,깔끔한 내부가 좋았습니다,2019. 11. 25,5.0,13,깔끔한 내부가 좋았습니다,0,2
3,아고다,나인트리 프리미어 호텔 명동 2,10.0,가격이 저렴해서 어딘가 아쉬운점은 한두개씩 있겠거니 하고 큰 기대없이 갔는데 생각외...,NaN,NaN,285,가격이 저렴해서 어딘가 아쉬운 점은 한두 개씩 있겠거니 하고 큰 기대 없이 갔는데 ...,1,2
4,아고다,나인트리 프리미어 호텔 명동 2,7.6,우선 뷰가 공사현장 보이고 주유소 보여서 그랬구요 새벽 6시쯤 넘어서는 공사장 소...,NaN,NaN,93,우선 뷰가 공사현장 보이고 주유소 보여서 그랬고요 새벽 6시쯤 넘어서는 공사장 소...,1,0


# Sentencepiece 적용

In [4]:
with open('review.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(review['review_spell_check']))

In [5]:
from sentencepiece import SentencePieceTrainer
sp = SentencePieceTrainer.Train('--input=review.txt --model_prefix=review --vocab_size=3000')

In [6]:
from sentencepiece import SentencePieceProcessor
sp = SentencePieceProcessor()
sp.Load("review.model")

True

In [7]:
sp.encode_as_pieces(review.loc[123, 'review'])

['▁여러',
 '지',
 '점',
 '▁가',
 '봤',
 '는데',
 '▁느낌',
 '은',
 '▁다',
 '▁비슷',
 '하고',
 '▁광화문점',
 '이',
 '▁서울',
 '여행',
 '하기',
 '에',
 '▁위치',
 '는',
 '▁좋은',
 '것',
 '같',
 '아요',
 '▁대신',
 '▁다',
 '른',
 '지',
 '점',
 '들',
 '보다',
 '▁조금',
 '은',
 '▁비싼',
 '▁',
 '감이',
 '▁있',
 '네',
 '요']

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(lowercase=False, tokenizer=sp.encode_as_pieces) #tokenizer를 안해주면, 띄어쓰기 단위로 잘라줌

In [10]:
tdm = cv.fit_transform(review['review_spell_check'])

# 훈련, 테스트 데이터 나누기 

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x = tdm
y = review['helpful']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
import joblib

In [15]:
cv.tokenizer = None # joblib로 저장하기 위해 필요

In [16]:
joblib.dump((cv, x_train, x_test, y_train, y_test), 'hotel_subword.pkl')

['hotel_subword.pkl']

In [17]:
cv, x_train, x_test, y_train, y_test = joblib.load('hotel_subword.pkl')

In [18]:
x_train.shape

(1974, 2996)

# 모형

In [19]:
import tensorflow as tf

In [20]:
model = tf.keras.models.Sequential()

In [21]:
model.add(tf.keras.layers.Dense(
    1,  #1,0 중 하나가 나온다! y자체는 열이 하나 ->Dense(1, )
    input_shape = (2996, ), #input data는 3018개 : 튶플로 넣어주기
    activation = 'sigmoid',
    kernel_regularizer = tf.keras.regularizers.l1_l2(0, 0.001) #ㅣ1은 0만, l2는 0.001만
))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 2997      
Total params: 2,997
Trainable params: 2,997
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy'])


In [32]:
model.fit(x_train.toarray(), y_train.values, epochs=100, validation_split=0.1,
          callbacks=[tf.keras.callbacks.EarlyStopping()])

Epoch 1/100
56/56 [==============================] - 2s 13ms/step - loss: 0.6719 - accuracy: 0.5976 - val_loss: 0.6259 - val_accuracy: 0.6919
Epoch 2/100
56/56 [==============================] - 0s 7ms/step - loss: 0.6040 - accuracy: 0.7105 - val_loss: 0.6012 - val_accuracy: 0.7273
Epoch 3/100
56/56 [==============================] - 0s 4ms/step - loss: 0.5805 - accuracy: 0.8178 - val_loss: 0.5837 - val_accuracy: 0.7879
Epoch 4/100
56/56 [==============================] - 0s 3ms/step - loss: 0.5654 - accuracy: 0.8499 - val_loss: 0.5709 - val_accuracy: 0.8131
Epoch 5/100
56/56 [==============================] - 0s 3ms/step - loss: 0.5350 - accuracy: 0.8580 - val_loss: 0.5592 - val_accuracy: 0.8232
Epoch 6/100
56/56 [==============================] - 0s 4ms/step - loss: 0.5144 - accuracy: 0.8745 - val_loss: 0.5496 - val_accuracy: 0.8232
Epoch 7/100
56/56 [==============================] - 0s 4ms/step - loss: 0.4930 - accuracy: 0.8873 - val_loss: 0.5415 - val_accuracy: 0.8283
Epoch 8/100


In [33]:
model.evaluate(x_test.toarray(), y_test.values, verbose=0) #손실, accuracy

[0.5522161722183228, 0.8036437034606934]

# 계수 확인

In [37]:
weights, _ = model.trainable_weights

In [38]:
token_weight = pd.DataFrame({'토큰': cv.get_feature_names(), '가중치': weights.numpy().flat})

In [39]:
token_weight.sort_values('가중치').head()

,토큰,가중치
1434,▁좋아요,-0.741736
337,▁깔끔,-0.447911
295,▁굿,-0.410221
1605,▁친절합니다,-0.371433
274,▁괜찮아,-0.368910


In [40]:
token_weight.sort_values('가중치').tail()

,토큰,가중치
1689,▁편의점,0.457330
738,▁방음,0.504998
193,▁객실,0.540762
398,▁넓,0.561374
851,▁빼고,0.578300
